In [1]:
%matplotlib inline

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle

from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import glob
import os

In [ ]:
!git clone https://github.com/aim56009/AI-MA_project.git

# Dataloader

Maybe later: 

in dataloader:  pad_colate ----> consider only unpaded data for loss in training

 pad to longest seq in batch ..- better sort after len before loading batches



In [3]:
batch_size = 1      # needs pading for > 1
PATH_TO_DATA = "AI-MA_project/pianoroll"
workers = 0

In [4]:
# DATALOADER V1
"""class MusicDataset(Dataset):

    def __init__(self, data_dir, labels, n, transforms=None):
        self.transforms = transforms
        self.data_dir = data_dir
        # load all files like below  (maybe do dicts instead of 1 big array)
        # make list of dict.  -voice_0 ..voice_all, length (-6 in total)


        for iLabel in range(len(labels)):
            file_names = sorted(glob.glob(os.path.join(data_dir, labels[iLabel], "*.pkl")))    

            if iLabel == 0:
                self.data = np.array(file_names[:n])
                self.labels = np.array(np.repeat(labels[iLabel], len(self.data)))
                
            else:
                self.data = np.append(self.data, file_names[:n])
                self.labels = np.append(self.labels, np.repeat(labels[iLabel], len(self.data)))
                


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):    ###maybe later revert sorted idx to file name idx
        print(self.data[idx])

        with open(self.data[idx]  ,'rb') as f:
          loaded_obj = pickle.load(f)
    
        pr_load = torch.tensor(loaded_obj.T)      # transpose np array first


        sample = {'pr': pr_load.float(), 'label': self.labels[idx]}     # also output the len 
        return sample
"""

In [114]:
class MusicDataset(Dataset):

    def __init__(self, data_dir, labels, n, transforms=None):
        self.transforms = transforms
        self.data_dir = data_dir

        self.labels = labels
        self.pr_dict = {}
                                                                                # load all files like below  (maybe do dicts instead of 1 big array)
                                                                                # make list of dict.  -voice_0 ..voice_all, length (-6 in total)
        len_list = []

        for iLabel in range(len(labels)):
            
            if iLabel == 4:   
                
                voice_files = []
                file_names = sorted(glob.glob(os.path.join(PATH_TO_DATA, self.labels[iLabel], "*.pkl")))   
                for name in file_names:
                    with open(name ,'rb') as f: 
                        loaded_obj = pickle.load(f)  
                        #loaded_obj = loaded_obj.T
                        voice_files.append(loaded_obj) 
                        len_list.append(len(loaded_obj.T))                             # for later: sort it after len(rows) when loading batches - to not have batches with len 1 and len 100 and then need to pad 99 zeros
                self.pr_dict[self.labels[iLabel]] = voice_files 
                self.pr_dict["length"] = len_list
            
            else:
                voice_files = []
                file_names = sorted(glob.glob(os.path.join(PATH_TO_DATA, self.labels[iLabel], "*.pkl")))   
                for name in file_names:
                    with open(name ,'rb') as f: 
                        loaded_obj = pickle.load(f)     
                        voice_files.append(loaded_obj)                              # for later: sort it after len(rows) when loading batches - to not have batches with len 1 and len 100 and then need to pad 99 zeros
                self.pr_dict[self.labels[iLabel]] = voice_files 
                                                    
        


    def __len__(self):
      
        return len(self.pr_dict[self.labels[0]])

    def __getitem__(self, idx):                                                 ###maybe later revert sorted idx to file name idx

        out_list = []
        for key, value in self.pr_dict.items():
            out_list.append(self.pr_dict[key][idx])
          
        v0 = torch.tensor(out_list[0].T)
        v1 = torch.tensor(out_list[1].T)
        v2 = torch.tensor(out_list[2].T)
        v3 = torch.tensor(out_list[3].T)
        v_all = torch.tensor(out_list[4].T) 

        length = self.pr_dict["length"]

        # v0.shape =torch.Size([816, 128])  , when doing v0, v1, v2, v3, v_all  = real , print(v0.shape) it is torch.Size([1, 756, 128])  - normal?

        return v0, v1, v2, v3, v_all, length

In [115]:
labels = ["voice_0", "voice_1", "voice_2", "voice_3", "voice_all"]
#labels = ["voice_all","voice_0"]
dataset = MusicDataset(PATH_TO_DATA, labels, 368)

In [116]:
loader = torch.utils.data.DataLoader(dataset,batch_size=batch_size, shuffle=False, num_workers=workers, drop_last=True)

### just for testing 

In [112]:
for batch_idx, real in enumerate(loader):
  v0, v1, v2, v3, v_all, length  = real
print(length)
  #print(v0.shape, v1.shape, v2.shape, v3.shape, v_all.shape)

[tensor([756]), tensor([624]), tensor([480]), tensor([480]), tensor([816]), tensor([384]), tensor([1332]), tensor([960]), tensor([576]), tensor([624]), tensor([1344]), tensor([576]), tensor([768]), tensor([672]), tensor([588]), tensor([816]), tensor([648]), tensor([624]), tensor([432]), tensor([576]), tensor([576]), tensor([720]), tensor([576]), tensor([576]), tensor([576]), tensor([480]), tensor([480]), tensor([384]), tensor([624]), tensor([564]), tensor([480]), tensor([576]), tensor([420]), tensor([624]), tensor([384]), tensor([672]), tensor([576]), tensor([480]), tensor([576]), tensor([384]), tensor([672]), tensor([432]), tensor([384]), tensor([624]), tensor([384]), tensor([576]), tensor([480]), tensor([588]), tensor([576]), tensor([480]), tensor([720]), tensor([576]), tensor([480]), tensor([480]), tensor([780]), tensor([396]), tensor([912]), tensor([528]), tensor([576]), tensor([768]), tensor([432]), tensor([576]), tensor([624]), tensor([480]), tensor([672]), tensor([624]), tensor(

In [113]:
v0.shape

torch.Size([1, 576, 128])

# Model - RNN

In [117]:
#https://github.com/fosfrancesco/pkspell/blob/main/src/models/models.py

In [118]:
class MusicRNN(nn.Module):

    def __init__(self, output_dim, input_dim=17, hidden_dim=300, rnn_depth=1, cell_type="GRU"):
        super(MusicRNN, self).__init__()

        self.n_out = output_dim
        
        
        rnn_cell = nn.GRU

        self.rnn = rnn_cell(
            input_size=input_dim,
            hidden_size= hidden_dim,
            num_layers=rnn_depth)


        # Output layers. The input will be two times
        self.top_layer_voice_0 = nn.Linear(hidden_dim, self.n_out)
        self.top_layer_voice_1 = nn.Linear(hidden_dim, self.n_out)
        self.top_layer_voice_2 = nn.Linear(hidden_dim, self.n_out)
        self.top_layer_voice_3 = nn.Linear(hidden_dim, self.n_out)

        # Loss function that we will use during training.
        self.loss = nn.CrossEntropyLoss(reduction="mean")


    def compute_outputs(self, sentences, sentences_len):

        # pack padded seq
        rnn_out = self.rnn(sentences)
        # pad packed seq

        out_0 = self.top_layer_voice_0(rnn_out)
        out_1 = self.top_layer_voice_1(rnn_out)
        out_2 = self.top_layer_voice_2(rnn_out)
        out_3 = self.top_layer_voice_3(rnn_out)
        

        return out_0, out_1, out_2, out_3

    def forward(self, sentences, v0, v1, v2, v3, sentences_len):

        # Compute the outputs. The shape is (max_len, n_sentences, n_labels).
        scores_0, scores_1, scores_2, scores_3 = self.compute_outputs(sentences, sentences_len)

        # Flatten the outputs and the labels, to compute the loss.
        # The input to this loss needs to be one 2-dimensional and one 1-dimensional tensor.
        score_0  = scores_0.view(-1, self.n_out)
        score_1  = scores_1.view(-1, self.n_out)
        score_2  = scores_2.view(-1, self.n_out)
        score_3  = scores_3.view(-1, self.n_out)

        # dont take padded len -> loss(score_0[:sentence_len], v0)      -> therefore store real len of voices  - before view

        v0 = v0.view(-1)
        v1 = v1.view(-1)
        v2 = v2.view(-1)
        v3 = v3.view(-1)
 
        
        loss = self.loss_pitch(score_0, v0) + self.loss_pitch(score_1, v1) + self.loss_pitch(score_2, v2) + self.loss_pitch(score_3, v3)
        
        return loss

    def predict(self, sentences, sentences_len):

        # Compute the outputs from the linear units.
        scores_0, scores_1, scores_2, scores_3 = self.compute_outputs(sentences, sentences_len)

        # Select the top-scoring labels. The shape is now (max_len, n_sentences).
        predicted_0 = scores_0.argmax(dim=2)
        predicted_1 = scores_1.argmax(dim=2)
        predicted_2 = scores_2.argmax(dim=2)
        predicted_3 = scores_3.argmax(dim=2)       # prbl distr over 0-not there, 1-there   - argmax -> take higher probl of those 2
        
        ##  predicted_X should be enough for BS = 1;  
        return (
            [
                predicted_0[: int(l), i].cpu().numpy()
                for i, l in enumerate(sentences_len)
            ],
            [
                predicted_1[: int(l), i].cpu().numpy()
                for i, l in enumerate(sentences_len)
            ],
            [
                predicted_2[: int(l), i].cpu().numpy()
                for i, l in enumerate(sentences_len)
            ],
            [
                predicted_3[: int(l), i].cpu().numpy()
                for i, l in enumerate(sentences_len)
            ],
        )



# Train

In [121]:
from torch import optim
from torch.optim import lr_scheduler

def train(epochs, lr, hidden_dim, momentum, rnn_depth, device, rnn_cell, weight_decay, train_dataloader, val_dataloader=None):
  
    
    model = MusicRNN(output_dim, input_dim, hidden_dim, rnn_depth, cell_type)

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = lr_scheduler.MultiStepLR(optimizer, [epochs // 2], gamma=0.1, verbose=True)


    history = training_loop(model, optimizer, train_dataloader, epochs=epochs, val_dataloader=val_dataloader, device=device, scheduler=scheduler)

    return model, history

In [119]:
import random
import click
import sklearn
import sklearn.model_selection
from collections import defaultdict
from sklearn.metrics import accuracy_score
from pathlib import Path
import sys

#sys.path.append(str(Path(__file__).resolve().parents[2]))

In [120]:
def training_loop(model,optimizer, train_dataloader, epochs=50, val_dataloader=None, device=None, scheduler=None,):
    if device is None:
        device = (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
        print(f"Training on device: {device}")

    # Move model to GPU if needed
    model = model.to(device)

    history = defaultdict(list)

    for i_epoch in range(1, epochs + 1):
        loss_sum = 0
        accuracy_pitch_sum = 0
        accuracy_ks_sum = 0
        model.train()

        for idx, (seqs, pitches, keysignatures, lens,) in enumerate(train_dataloader):  # seqs, pitches, keysignatures, lens are batches

            seqs, pitches, keysignatures = (seqs.to(device), pitches.to(device),keysignatures.to(device),)
            optimizer.zero_grad()
            loss = model(seqs, pitches, keysignatures, lens)
            loss.backward()
            optimizer.step()
            loss_sum += loss.item()

            with torch.no_grad():
                predicted_pitch, predicted_ks = model.predict(seqs, lens)
                for i, (p, k) in enumerate(zip(predicted_pitch, predicted_ks)):
                    # compute the accuracy without considering the padding
                    acc_pitch = accuracy_score(p, pitches[:, i][: len(p)].cpu())
                    acc_ks = accuracy_score(k, keysignatures[:, i][: len(k)].cpu())
                    # normalize according to the number of sequences in the batch
                    accuracy_pitch_sum += acc_pitch / len(lens)
                    accuracy_ks_sum += acc_ks / len(lens)

        train_loss = loss_sum / len(train_dataloader)
        # normalize according to the number of batches
        train_accuracy_pitch = accuracy_pitch_sum / len(train_dataloader)
        train_accuracy_ks = accuracy_ks_sum / len(train_dataloader)
        history["train_loss"].append(train_loss)
        history["train_accuracy_pitch"].append(train_accuracy_pitch)
        history["train_accuracy_ks"].append(train_accuracy_ks)
        
        print("Train Loss: {}, Train Accuracy (Pitch): {}, Train Accuracy (KS): {}".format(
                train_loss, train_accuracy_pitch, train_accuracy_ks))

        if val_dataloader is not None:
            # Evaluate on the validation set
            model.eval()
            accuracy_pitch_sum = 0
            accuracy_ks_sum = 0
            with torch.no_grad():
                for seqs, pitches, keysignatures, lens in val_dataloader:
                    # Predict the model's output on a batch
                    predicted_pitch, predicted_ks = model.predict(seqs.to(device), lens)
                    # Update the lists that will be used to compute the accuracy
                    for i, (p, k) in enumerate(zip(predicted_pitch, predicted_ks)):
                        # compute the accuracy without considering the padding
                        acc_pitch = accuracy_score(p, pitches[:, i][: len(p)].cpu())
                        acc_ks = accuracy_score(k, keysignatures[:, i][: len(k)].cpu())
                        # normalize according to the number of sequences in the batch
                        accuracy_pitch_sum += acc_pitch / len(lens)
                        accuracy_ks_sum += acc_ks / len(lens)

                # normalize according to the number of batches
                val_accuracy_pitch = accuracy_pitch_sum / len(val_dataloader)
                val_accuracy_ks = accuracy_ks_sum / len(val_dataloader)

            history["val_accuracy_pitch"].append(val_accuracy_pitch)
            history["val_accuracy_ks"].append(val_accuracy_ks)
            
            print("Validation Accuracy (Pitch): {}, Validation Accuracy (KS): {}".format(
                    val_accuracy_pitch, val_accuracy_ks))

        if scheduler is not None:
            scheduler.step()

        # save the model
        torch.save(model, Path("./models/temp/model_temp_epoch{}.pkl".format(i_epoch)))

    return history

In [122]:
def start_experiment(
    model,
    epochs,
    lr,
    hidden_dim,
    bs,
    momentum,
    hidden_dim2,
    rnn_depth,
    device,
    dropout,
    dropout2,
    cell,
    decay,
    optimizer,
    learn_all,
    bidirectional,
    mode,
    augmentation):
    
    print("Loading the augmented ASAP dataset")
    # load the asap datasets with ks
    with open(Path("./data/processed/asap_augmented.pkl"), "rb") as fid:
        full_list_of_dict_dataset = pickle.load(fid)

    paths = list(set([e["original_path"] for e in full_list_of_dict_dataset]))
    list_of_dict_dataset = keep_best_transpositions(full_list_of_dict_dataset)

    # remove pieces from asap that are in Musedata
    paths = [p for p in paths if p != "Bach/Prelude/bwv_865/xml_score.musicxml"]
    # remove mozart Fantasie because of incoherent key signature
    paths = [p for p in paths if p != "Mozart/Fantasie_475/xml_score.musicxml"]
    paths = sorted(paths)

    from functools import partial

    trainer = partial(
        train_pkspell,
        model,
        epochs,
        lr,
        hidden_dim,
        momentum,
        hidden_dim2,
        rnn_depth,
        device,
        dropout,
        dropout2,
        cell,
        decay,
        optimizer,
        bidirectional,
        mode)

    def train_dataloader(ds):
        return DataLoader(ds, batch_size=bs, shuffle=True, collate_fn=pad_collate, num_workers=2)

    def val_dataloader(ds):
        return DataLoader(ds, batch_size=1, shuffle=False, collate_fn=pad_collate, num_workers=1)

    if learn_all:
        print("Learning from full dataset")
        train_dataset = PSDataset(
            list_of_dict_dataset,
            paths,
            transform_pc,
            transform_tpc,
            transform_key,
            augment_dataset=augmentation,
            sort=True,
            truncate=None)
        
        _, history = trainer(train_dataloader(train_dataset),)
    else:
        paths = np.array(paths)
        # Divide train and validation set
        path_train, path_validation = sklearn.model_selection.train_test_split(
            paths, test_size=0.15, random_state=seed,
        )
        print("Train and validation lenghts: ", len(path_train), len(path_validation))
        train_dataset = PSDataset(
            list_of_dict_dataset,
            path_train,
            transform_pc,
            transform_tpc,
            transform_key,
            augment_dataset=augmentation,
            sort=True,
            truncate=None) 
        
        validation_dataset = PSDataset(
            list_of_dict_dataset,
            path_validation,
            transform_pc,
            transform_tpc,
            transform_key,
            augment_dataset=False)
        
        _, history = trainer(
            train_dataloader(train_dataset), val_dataloader(validation_dataset))